In [ ]:
%pip install -q "unstructured[csv,xlsx,pdf]" "astrapy" "openai" "numpy" "cassandra-driver" "html2text"


In [1]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/Shareddrives/Field Engineering/Customer Documentation/F/Fiutur/poc/01 SMART Methane Project Registration Samples"
path2 = "/content/drive/Shareddrives/Field Engineering/Customer Documentation/F/Fiutur/poc/01 SMART Methane Project Registration Samples/Production Data"

Mounted at /content/drive


In [2]:
!ls "/content/drive/Shareddrives/Field Engineering/Customer Documentation/F/Fiutur/poc/01 SMART Methane Project Registration Samples"

'BPX Gas Quality Analysis - 20190723.xlsx'
'BPX Haynesville Emission Workbook 20190919.xlsx'
'BPX Haynesville Facility P_ID.pdf'
'BPX Haynesville OGMP-2.0-Upstream_Reporting_Templat_MOCK.xlsx'
 pdf-images
'Production Data'


In [4]:
import glob
files1 = glob.glob(path+"/*.xlsx")
files2 = glob.glob(path2+"/*.csv")
files = files1+files2

In [ ]:
from unstructured.partition.auto import partition
import html2text
import json

def to_astradb(el):
    return {
          "source": el.metadata.filename,
          "page": el.metadata.page_name,
          "number": el.metadata.page_number,
          "text": el.text.replace("'",""),
          "render": html2text.html2text(el.metadata.text_as_html.replace("'",""))
    }

print("Starting to parse data: ", end="")
jsons = []
for file in files:
    for element in partition(filename=file):
        print(".", end="", flush=True)
        if element.metadata.text_as_html is not None:
            jsons.append(to_astradb(element))

print("Finished parsing data", end="")


In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(jsons, orient='columns')
df = df.dropna()
cols = ['source', 'page', 'render']
df["combined"] = df[cols].apply(lambda row: '\n'.join(row.values.astype(str)), axis=1)
df.info()


In [9]:
df.sample(5)

,source,page,number,text,render,combined
46,BPX Haynesville Emission Workbook 20190919.xlsx,BPX EPA CO2ef,6.0,\n\n\n7.0\npounds /Metric Tonne - 1MT = 220462...,7.0 | pounds /Metric Tonne - 1MT = 220462 \n-...,BPX Haynesville Emission Workbook 20190919.xls...
3,BPX Haynesville OGMP-2.0-Upstream_Reporting_Te...,"L3, L4 and L5 Assets",2.0,\n\n\nBP Bandicoot Drilling Unit 1HB\n42-405-3...,BP Bandicoot Drilling Unit 1HB | 42-405-30585-...,BPX Haynesville OGMP-2.0-Upstream_Reporting_Te...
30,BPX Haynesville Emission Workbook 20190919.xlsx,BPX EPA CO2ef,6.0,\n\n\nSep Outlet to meter\nValve\n19\n0.121\n2...,Sep Outlet to meter | Valve | 19 | 0.121 | 2.2...,BPX Haynesville Emission Workbook 20190919.xls...
10,BPX Haynesville Emission Workbook 20190919.xlsx,BPX Daily,1.0,\n\n\nTotal mmbtu per day\n80487.22964\n\n\nBO...,Total mmbtu per day | 80487.22964 \n---|--- ...,BPX Haynesville Emission Workbook 20190919.xls...
31,BPX Haynesville Emission Workbook 20190919.xlsx,BPX EPA CO2ef,6.0,\n\n\nSales Meter\nValve\n8\n0.121\n0.968\n\n\...,Sales Meter | Valve | 8 | 0.121 | 0.968 | | ...,BPX Haynesville Emission Workbook 20190919.xls...


In [ ]:
for a,b in df.iterrows():
    print("----------------------------- New Table ------------------------")
    print (b.combined.replace('\n\n',' '))

In [ ]:
# Connecting to Astra requires a secure connect bundle
# upload secure connect bundle
try:
    from google.colab import files
    IN_COLAB = True
except:
    IN_COLAB = False
import os

if IN_COLAB:
    uploaded = files.upload()
    if uploaded:
        astraBundleFileTitle = list(uploaded.keys())[0]
        ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
    else:
        raise ValueError(
            'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
        )
else:
    # if running in jupyter lab set path manually
    ASTRA_DB_SECURE_BUNDLE_PATH = None

In [9]:
from google.colab import userdata

id = userdata.get("CLIENT_ID")
secret = userdata.get("CLIENT_SECRET")

### Connect via CQL

In [20]:

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Configure cloud connection
cloud_config = {
    'secure_connect_bundle': ASTRA_DB_SECURE_BUNDLE_PATH
}

# Create authentication provider
auth_provider = PlainTextAuthProvider(id, secret)

# Connect to Cassandra cluster
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
# Create session
session = cluster.connect()
# Execute query
row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
    a = session
else:
    print("An error occurred.")


ERROR:cassandra.connection:Closing connection <LibevConnection(140348904391200) 9f3cd1e5-5d53-492d-9973-3bd90c0e5234-us-east-2.db.astra.datastax.com:29042:d154bc04-52e5-49f8-a12c-99b550924b57> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-09ec37c912ed


### OpenAI

In [25]:
from openai import Client
client = Client(api_key=userdata.get("OPENAI_API_KEY"))


### Embedding via Data API

In [26]:
BATCH_SIZE = 20
MODEL_NAME = "text-embedding-3-small"

In [17]:
import numpy as np

print("Starting to vectorize data ", end="")

vectors = []
# Loop BATCH_SIZE at the time
for k,g in df.groupby(np.arange(len(df))//BATCH_SIZE):
    # compute the embedding vectors for this batch
    for vector in client.embeddings.create(input=g.combined, model=MODEL_NAME).data:
        vectors.append(vector.embedding)
        print(".", end="", flush=True)

vector_series  = pd.Series(vectors, name="vector")
print("Finished vectorizing entries.")


Starting to vectorize data .............................................................Finished vectorizing entries.


In [18]:
vector_series.shape


(61,)

In [24]:
cql = "use default_keyspace;"
session.execute(cql)


In [ ]:
cql = """DROP TABLE well_data;"""
session.execute(cql)

In [ ]:
cql = """CREATE TABLE IF NOT EXISTS well_data (
        id INT PRIMARY KEY,
        source TEXT,
        page TEXT,
        number FLOAT,
        text TEXT,
        render TEXT,
        vector VECTOR <FLOAT, 1536>
);"""
session.execute(cql)


In [ ]:
cql = """CREATE INDEX IF NOT EXISTS vector_index ON well_data(vector);"""
session.execute(cql)


In [ ]:
cql = "truncate table well_data"
session.execute(cql)


In [25]:
insert_statements = []
for line, vector in zip(df.iterrows(), vectors):
    indx = line[0]
    row = line[1]
    insert_statement = f"""INSERT INTO well_data (id, source, page, number, text, render, vector) VALUES ({indx}, '{row['source']}', '{row['page']}', {row['number']}, '{row['text']}', '{row['render']}', {vector});"""
    insert_statements.append(insert_statement)


In [ ]:
insert_statement[:1000]


In [ ]:
print("Starting to save data ", end="")
for insert_statement in insert_statements:
    print(".", end="", flush=True)
    session.execute(insert_statement)

print("Finished saving data", end="")


### Vector Search

In [ ]:
question = "Considering a harmonic production decline rate seen from the two months of data, predict the Carbon Intensity at April 4, 2020 for Bandicoot 2h in terms of Tonnes of CO2e / BOE"
question_vector = client.embeddings.create(input=question,model=MODEL_NAME)

cql = """SELECT  source, text, render, similarity_cosine(vector, {vector})
    FROM well_data
    ORDER BY vector ANN OF {vector} LIMIT 10;
""".format(vector=question_vector.data[0].embedding)
rows = session.execute(cql)
tables = '\n'.join(html2text.html2text(row.text) for row in rows)
print(tables)

In [44]:
persona = """You are an expert analyst in Oil and Gas Industry with focus on Environmental Impact Analysis. You have intense knowledge of how well production works. you are extremely familiar with the operations, setup, machines and how to calculate output etc of these wells.
As part of this job you are examining the environmental impact of well operations on the surrounding environment. You are particularly interested in assessing factors such as air and water pollution, habitat disruption, and overall ecosystem health.

To conduct this analysis, you will be supplied with actual raw production numbers from well. based on this data please answer user question. and please try to be as elaborative as possible and explain your reasoning. Please also quote the source of your information.
"""

In [45]:
print(client.chat.completions.create(
        messages=[
            {"role": "system", "content": persona},
            {"role": "user", "content": tables},
            {"role": "user", "content": question}
        ],
        model="gpt-4",
        temperature=0.1
    ).choices[0].message.content)

To predict the Carbon Intensity (CI) for Bandicoot 2h at April 4, 2020, we need to understand the concept of Carbon Intensity and the factors that influence it. Carbon Intensity is a measure of how much carbon (CO2) emissions are produced per unit of oil and gas produced (BOE - Barrel of Oil Equivalent). It is typically measured in Tonnes of CO2e (CO2 equivalent) per BOE.

The Carbon Intensity can be influenced by several factors including the type of hydrocarbon being produced (oil, gas, or condensate), the production method (conventional or unconventional), the age and depth of the well, the amount of flaring and venting, and the efficiency of the equipment used in production.

In the data provided, we don't have a direct measure of the Carbon Intensity for Bandicoot 2h. However, we do have data on the CO2 emissions and the Mean Daily Production in mcfd (thousand cubic feet per day). 

To predict the Carbon Intensity for Bandicoot 2h at April 4, 2020, we would need to make some assum